# Scrape Linkedin Data

Crawling Linkedin is against the user agreement of Linkedin! This is used for educational purpose only.

In [1]:
# Make sure we have installed the dependency
! pip freeze | grep linkedin

linkedin-scraper==2.11.2


In [2]:
! google-chrome-stable --version

Google Chrome 114.0.5735.90 


In [3]:
from linkedin_scraper import JobSearch, Job, actions
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import os
from pprint import pprint

def set_chrome_options() -> Options:
    """Sets chrome options for Selenium.
    Chrome options for headless browser is enabled.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_prefs = {}
    chrome_options.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    return chrome_options

def scrape_job_search(keyword):
    driver = webdriver.Chrome(options=set_chrome_options())
    actions.login(driver, os.environ["EMAIL"], os.environ["PWORD"]) # if email and password isnt given, it'll prompt in terminal
    print("... Logged in.")
    job_search = JobSearch(driver=driver, close_on_complete=False, scrape=False)

    job_listings = job_search.search(keyword) # returns the list of `Job` from the first page
    return job_listings

def scrape_job(job_link):
    driver = webdriver.Chrome(options=set_chrome_options())
    actions.login(driver, os.environ["EMAIL"], os.environ["PWORD"]) # if email and password isnt given, it'll prompt in terminal
    job = Job(job_link, driver=driver, close_on_complete=False)
    return job

## 2. Crawl job listings

### Test run

In [25]:
_job_listings = scrape_job_search("data")
_job_listings

[<Job Senior Data Analyst, Ads Rovio Entertainment Corporation>,
 <Job Senior Game Analyst Next Games, A Netflix Game Studio>,
 <Job Azure Data Architect Cloud1 Oy>,
 <Job Data Architect Nortal>,
 <Job Data Science - Machine Learning Engineer Wolt>,
 <Job Data Engineer The Hub>,
 <Job Machine learning expert Verso Vision>,
 <Job Data Scientist / Senior Data Scientist Basware>,
 <Job Data Engineer Schibsted Finland>,
 <Job Data Scientist, Pohjola Vakuutuksen analytiikka, Helsinki OP Financial Group>,
 <Job AI Engineer Academy - Finland Avanade>,
 <Job Cloud Data Engineer Sweco>,
 <Job Data Scientist, Pohjola Insurance Analytics, Helsinki OP Financial Group>,
 <Job AI Data Engineer Greenstep>]

In [20]:
vars(_job_listings[1])

{'driver': <selenium.webdriver.chrome.webdriver.WebDriver (session="b22d22debbaefb1f4feed3e1aa195604")>,
 'linkedin_url': 'https://www.linkedin.com/jobs/view/3717037977/?eBP=CwEAAAGLw9ALyc5DCqJDfvQR9Je-pv7uBp7zkqbMf7DZ9IlglaBTKuhEqNapgn6XA8-_69gkbrNgR3yU9kJe4erJhOwwhX7KBGfbkAoz2F3QIx6A1z2tlyTTMw2xSvlqGYE84bcM9rNgMTcIF-Q1qwwCWognBYgiF2lklVGg823k6sa_7-p0nNn0VeEWf8i7jzfM3L9DaoGN-ZF_G5w2b4Osq-h6IUkL8xmQXyMq_l9w5VrcfWG1WCkDytG9oQEhij3oLgMTI7Wief7uq95XNbbFzfqH1x-Kb1VUM_RtDLsmqWAFkwqnQV2-m0fdyTK4pPhAhIICvznEP7CUSAM7OJ78bF3ilpsrzk53u4NWpjqEca-LeWyNssq9dDPBCVRFy5gDlVROB3njzZKMEFXOCkiT2fmGsMN3zJFaHg&refId=bkXKrgYl%2FzIWkBcDYuyykw%3D%3D&trackingId=Oi%2F2zJY8cXpPQGa6xWnAhQ%3D%3D&trk=flagship3_search_srp_jobs',
 'job_title': 'Senior Game Analyst',
 'company': 'Next Games, A Netflix Game Studio',
 'company_linkedin_url': None,
 'location': 'Helsinki, Uusimaa, Finland (Hybrid)',
 'posted_date': None,
 'applicant_count': None,
 'job_description': None,
 'benefits': None}

In [29]:
_job_listings[0].WAIT_FOR_ELEMENT_TIMEOUT = 30

In [30]:
# This doesn't work: Timeout Error
# _job_listings[0].scrape(close_on_complete=False)
# vars(_job_listings[0])

In [8]:
# Save the crawled webpage
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# with open("job_page_example.html", "w") as f:
#     f.write(_job_listings[0].driver.page_source)

In [17]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

# job_title
WebDriverWait(_job_listings[1].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'jobs-unified-top-card__job-title')]"
        )
    )
).text.strip()

'Senior Data Analyst, Ads'

In [18]:
# company
WebDriverWait(_job_listings[1].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//a[1]"
        )
    )
).text.strip()

'Rovio Entertainment Corporation'

In [19]:
# Location
WebDriverWait(_job_listings[1].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//*"
        )
    )
).text.strip()

'Rovio Entertainment Corporation · Helsinki, Uusimaa, Finland Reposted  3 weeks ago  · Over 50 applicants'

In [34]:
# Job type
WebDriverWait(_job_listings[0].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'ui-label ui-label--accent-3 text-body-small')]/span"
        )
    )
).text.strip()

'Hybrid'

In [21]:
# Job type 2
WebDriverWait(_job_listings[1].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "(//*[contains(@class, 'ui-label ui-label--accent-3 text-body-small')])[2]//span"
        )
    )
).text.strip()

'Full-time'

In [22]:
# Post Date
WebDriverWait(_job_listings[1].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//span[3]"
        )
    )
).text.strip()

'Reposted  3 weeks ago'

In [23]:
# company linkedin url
WebDriverWait(_job_listings[1].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//a"
        )
    )
).get_attribute("href")

'https://www.linkedin.com/company/rovio/life'

In [31]:
# Job description
job_description_elem = WebDriverWait(_job_listings[1].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'jobs-description')]"
        )
    )
)

job_description_elem.text.strip()

'About the job\nAt Rovio you will get to work with one of the most famous games IP’s in the world: Angry Birds. Our mission is to “craft joy” for the wide audience that we reach with our games and products. In order to do that, we know that people need to bring their own joy to what we do. That’s why we value work-life balance, say no to crunch culture, and welcome people from all walks of life to join the flock. Today, we are a proud team of 500+ caring and talented professionals representing 54 different nations.\n\nWe trust our teams to work autonomously by providing them the right tools and level of responsibility. We believe in our teams to remain creative and to keep learning – as well as ensuring everyone has opportunities for personal growth.\n\nData is at the heart of everything we do at Rovio. It enables us to continually improve our games and provide incredible experiences for the millions of users who play our games every day. All of this data is only valuable if it can be 

In [30]:
# First set of skills added by the job poster
WebDriverWait(_job_listings[1].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'job-details-how-you-match__skills-item')][1]//a"
        )
    )
).text

'Data Analysis, Python (Programming Language), and SQL'

In [18]:
# 2nd set of skills added by the job poster
WebDriverWait(_job_listings[0].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'job-details-how-you-match__skills-item')][2]//a"
        )
    )
).text

'Advertising, Communication, Dashboards, Gameplay, Key Metrics, Knowledge Acquisition, and Teamwork'

In [28]:
# Salary range and benefits
WebDriverWait(_job_listings[1].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'salary-main-rail-card')]"
        )
    )
).text.strip()

''

In [29]:
# Seniority
WebDriverWait(_job_listings[1].driver, 3).until(
    EC.presence_of_element_located(
        (By.XPATH,
        "//*[contains(@class, 'job-details-jobs-unified-top-card__job-insight-view-model-secondary')][2]"
        )
    )
).text.strip()

TimeoutException: Message: 
Stacktrace:
#0 0x5583a58504e3 <unknown>
#1 0x5583a557fc76 <unknown>
#2 0x5583a55bbc96 <unknown>
#3 0x5583a55bbdc1 <unknown>
#4 0x5583a55f57f4 <unknown>
#5 0x5583a55db03d <unknown>
#6 0x5583a55f330e <unknown>
#7 0x5583a55dade3 <unknown>
#8 0x5583a55b02dd <unknown>
#9 0x5583a55b134e <unknown>
#10 0x5583a58103e4 <unknown>
#11 0x5583a58143d7 <unknown>
#12 0x5583a581eb20 <unknown>
#13 0x5583a5815023 <unknown>
#14 0x5583a57e31aa <unknown>
#15 0x5583a58396b8 <unknown>
#16 0x5583a5839847 <unknown>
#17 0x5583a5849243 <unknown>
#18 0x7f66d4294ac3 <unknown>


In [53]:
driver = webdriver.Chrome(options=set_chrome_options())
actions.login(driver, os.environ["EMAIL"], os.environ["PWORD"]) # if email and password isnt given, it'll prompt in terminal
# job = _Job(linkedin_url="https://www.linkedin.com/jobs/collections/recommended/?currentJobId=3736532279", 
#            driver=driver, close_on_complete=False, scrape=True)

## Overwrite the crawler methods to debug

In [3]:
import logging
from linkedin_scraper import Job

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class _Job(Job):
    def __init__(self, **kwargs):
       self.job_title = ""
       self.required_skills = ""
       self.job_type_1 = ""
       self.job_type_2 = ""
 
       super().__init__(**kwargs)
    
    def scrape_logged_in(self, close_on_complete=True):
        driver = self.driver
        
        driver.get(self.linkedin_url)
        self.focus()
        self.job_title = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'jobs-unified-top-card__job-title')]").text.strip()
        self.company = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//a[1]").text.strip()
        self.company_linkedin_url = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//a").get_attribute("href")
        self.location = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//*").text.strip()
        self.posted_date = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-jobs-unified-top-card__primary-description')]//span[3]").text.strip()
        self.job_type_1 = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'ui-label ui-label--accent-3 text-body-small')]/span").text.strip()

        job_description_elem = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'jobs-description')]")
        inner_html = job_description_elem.get_attribute(name="innerHTML")
        soup = BeautifulSoup(inner_html, 'html.parser')
        
        self.job_description = '\n'.join(
            elem.get_text() for elem in soup.find_all() if elem.name in ["span", "ul", "strong", "li"]
        )
        
        try:
            self.required_skills = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-how-you-match__skills-item')][1]//a").text.strip()
        except TimeoutException as e:
            logger.error(str(e))

        try:
            self.required_skills += self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'job-details-how-you-match__skills-item')][2]//a").text.strip()
        except TimeoutException as e:
            logger.error(str(e))

        try:
            self.job_type_2 = self.wait_for_element_to_load(by=By.XPATH, name="(//*[contains(@class, 'ui-label ui-label--accent-3 text-body-small')])[2]/span").text.strip()
        except TimeoutException:
            self.job_type_2 = ""
            
        try:
            self.applicant_count = self.wait_for_element_to_load(by=By.XPATH, name="jobs-unified-top-card__applicant-count").text.strip()
        except TimeoutException:
            self.applicant_count = 0
        
        try:
            self.benefits = self.wait_for_element_to_load(by=By.XPATH, name="//*[contains(@class, 'salary-main-rail-card')]").text.strip()
        except TimeoutException:
            self.benefits = ""

        if close_on_complete:
            driver.close()

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from linkedin_scraper import actions

import os

def set_chrome_options() -> Options:
    """Sets chrome options for Selenium.
    Chrome options for headless browser is enabled.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_prefs = {}
    chrome_options.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    return chrome_options

def _scrape_job(job_link):
    driver = webdriver.Chrome(options=set_chrome_options())
    actions.login(driver, os.environ["EMAIL"], os.environ["PWORD"]) # if email and password isnt given, it'll prompt in terminal
    job = _Job(linkedin_url=job_link, driver=driver, close_on_complete=False, scrape=True)
    return job

_test_job = _scrape_job('https://www.linkedin.com/jobs/view/3702242885')

In [5]:
vars(_test_job)

{'job_title': 'Senior Data Analyst, Ads',
 'required_skills': 'Data Analysis, Python (Programming Language), and SQLAdvertising, Communication, Dashboards, Gameplay, Key Metrics, Knowledge Acquisition, and Teamwork',
 'job_type_1': 'Hybrid',
 'job_type_2': 'Full-time',
 'driver': <selenium.webdriver.chrome.webdriver.WebDriver (session="a81a6002b39ce9765c484e23ba0ecc43")>,
 'linkedin_url': 'https://www.linkedin.com/jobs/view/3702242885',
 'company': 'Rovio Entertainment Corporation',
 'company_linkedin_url': 'https://www.linkedin.com/company/rovio/life',
 'location': 'Rovio Entertainment Corporation · Helsinki, Uusimaa, Finland Reposted  3 weeks ago  · Over 50 applicants',
 'posted_date': 'Reposted  3 weeks ago',
 'applicant_count': 0,
 'job_description': '\nAt Rovio you will get to work with one of the most famous games IP’s in the world: Angry Birds. Our mission is to “craft joy” for the wide audience that we reach with our games and products. In order to do that, we know that people 

In [41]:
# _test_job.job_description_elem.get_attribute(name="innerHTML")

'\n\n<!---->\n      <article class="jobs-description__container\n          jobs-description__container--condensed">\n        <div class="jobs-description__content jobs-description-content\n            jobs-description__content--condensed">\n          <div class="jobs-box__html-content jobs-description-content__text t-14 t-normal\n              jobs-description-content__text--stretch" id="job-details" tabindex="-1">\n            <h2 class="text-heading-large\n                mb4">\n              About the job\n            </h2>\n\n<!---->\n<!---->            <span>\n                <!---->At Rovio you will get to work with one of the most famous games IP’s in the world: Angry Birds. Our mission is to “craft joy” for the wide audience that we reach with our games and products. In order to do that, we know that people need to bring their own joy to what we do. That’s why we value work-life balance, say no to crunch culture, and welcome people from all walks of life to join the flock. Toda

In [42]:
inner_html = _test_job.job_description_elem.get_attribute(name="innerHTML")
from bs4 import BeautifulSoup

In [44]:
soup = BeautifulSoup(inner_html, 'html.parser')
soup.get_text()

'\n\n\n\n\n\n              About the job\n            \n\n \nAt Rovio you will get to work with one of the most famous games IP’s in the world: Angry Birds. Our mission is to “craft joy” for the wide audience that we reach with our games and products. In order to do that, we know that people need to bring their own joy to what we do. That’s why we value work-life balance, say no to crunch culture, and welcome people from all walks of life to join the flock. Today, we are a proud team of 500+ caring and talented professionals representing 54 different nations.We trust our teams to work autonomously by providing them the right tools and level of responsibility. We believe in our teams to remain creative and to keep learning – as well as ensuring everyone has opportunities for personal growth.Data is at the heart of everything we do at Rovio. It enables us to continually improve our games and provide incredible experiences for the millions of users who play our games every day. All of thi

In [54]:
for elem in soup.find_all():
    if elem.name in ["span", "ul", "strong", "li"]:
        print(elem.name)
        print(elem.get_text())

span

At Rovio you will get to work with one of the most famous games IP’s in the world: Angry Birds. Our mission is to “craft joy” for the wide audience that we reach with our games and products. In order to do that, we know that people need to bring their own joy to what we do. That’s why we value work-life balance, say no to crunch culture, and welcome people from all walks of life to join the flock. Today, we are a proud team of 500+ caring and talented professionals representing 54 different nations.We trust our teams to work autonomously by providing them the right tools and level of responsibility. We believe in our teams to remain creative and to keep learning – as well as ensuring everyone has opportunities for personal growth.Data is at the heart of everything we do at Rovio. It enables us to continually improve our games and provide incredible experiences for the millions of users who play our games every day. All of this data is only valuable if it can be used to guide our 

In [57]:
print(
    '\n'.join(elem.get_text() for elem in soup.find_all() if elem.name in ["span", "ul", "strong", "li"])
     )


At Rovio you will get to work with one of the most famous games IP’s in the world: Angry Birds. Our mission is to “craft joy” for the wide audience that we reach with our games and products. In order to do that, we know that people need to bring their own joy to what we do. That’s why we value work-life balance, say no to crunch culture, and welcome people from all walks of life to join the flock. Today, we are a proud team of 500+ caring and talented professionals representing 54 different nations.We trust our teams to work autonomously by providing them the right tools and level of responsibility. We believe in our teams to remain creative and to keep learning – as well as ensuring everyone has opportunities for personal growth.Data is at the heart of everything we do at Rovio. It enables us to continually improve our games and provide incredible experiences for the millions of users who play our games every day. All of this data is only valuable if it can be used to guide our day t

In [11]:
# Save the crawled webpage
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

with open("job_page_example.html", "w") as f:
    f.write(_test_job.driver.page_source)

## Debug JobSearch to make it scrape new pages

In [5]:
from linkedin_scraper import JobSearch

driver = webdriver.Chrome(options=set_chrome_options())
actions.login(driver, os.environ["EMAIL"], os.environ["PWORD"]) # if email and password isnt given, it'll prompt in terminal
print("... Logged in.")
job_search = JobSearch(driver=driver, close_on_complete=False, scrape=False)

... Logged in.


In [9]:
import urllib
url = os.path.join(job_search.base_url, "search") + f"?keywords={urllib.parse.quote('data')}&refresh=true"

In [10]:
job_search.driver.get(url)

In [11]:
# Final redirection
job_search.driver.current_url

'https://www.linkedin.com/jobs/search/?currentJobId=3717037977&keywords=data&refresh=true'

In [7]:
from linkedin_scraper import JobSearch, Job, actions
from typing import List
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import os
from pprint import pprint
import urllib
from time import sleep

def set_chrome_options() -> Options:
    """Sets chrome options for Selenium.
    Chrome options for headless browser is enabled.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_prefs = {}
    chrome_options.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    return chrome_options

class _JobSearch(JobSearch):
    def __init__(self, final_url=None, **kwargs):
        self.final_url = final_url
        self.current_url = None
        super().__init__(**kwargs)
    
    def search(self, search_term: str, page_n) -> List[Job]:
        if self.final_url is None:
            self.current_url = os.path.join(self.base_url, "search") + f"?keywords={urllib.parse.quote(search_term)}&refresh=true"
            self.driver.get(self.current_url)

            # Get redirection URL
            self.final_url = self.driver.current_url
        else:
            self.current_url = os.path.join(self.final_url, f"&start={25*(page_n-1)}")
            self.driver.get(self.current_url)
        
        self.scroll_to_bottom()
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        job_listing_class_name = "jobs-search-results-list"
        job_listing = self.wait_for_element_to_load(name=job_listing_class_name)

        self.scroll_class_name_element_to_page_percent(job_listing_class_name, 0.3)
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        self.scroll_class_name_element_to_page_percent(job_listing_class_name, 0.6)
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        self.scroll_class_name_element_to_page_percent(job_listing_class_name, 1)
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        job_results = []
        for job_card in self.wait_for_all_elements_to_load(name="job-card-list", base=job_listing):
            job = self.scrape_job_card(job_card)
            job_results.append(job)
        return job_results

def _scrape_job_search(final_url, keyword, page_n):
    driver = webdriver.Chrome(options=set_chrome_options())
    actions.login(driver, os.environ["EMAIL"], os.environ["PWORD"]) # if email and password isnt given, it'll prompt in terminal
    print("... Logged in.")
    job_search = _JobSearch(driver=driver, close_on_complete=False, scrape=False)

    job_listings = job_search.search(keyword, page_n) # returns the list of `Job` from the first page
    return job_search, job_listings

In [8]:
_test_job_search, _test_listings = _scrape_job_search("data", page_n=1)
_test_listings

... Logged in.


[<Job Legal & Compliance Officer Sambla>,
 <Job Senior Game Analyst Next Games, A Netflix Game Studio>,
 <Job Data Analyst, Merge Mansion Metacore>,
 <Job Data Scientist, Data Platform & AI OP Financial Group>,
 <Job Data Science - Machine Learning Engineer Wolt>,
 <Job Data Engineer The Hub>,
 <Job Senior Threat Intelligence Officer, Nordic Nordea>,
 <Job RWE Scientist / Epidemiologist MedEngine>,
 <Job Data Platform Engineer COR Group Oy>,
 <Job Data Engineer Sievo>,
 <Job Senior Data Analyst, Ads Rovio Entertainment Corporation>,
 <Job Data & AI Fin Informatica MDM Accenture Nordics>,
 <Job Senior Legal and Privacy Counsel, Veho Group Veho Oy Ab>,
 <Job Data Engineer The Hub>]

In [13]:
_test_listings_2 = _test_job_search.search("data", page_n=2)
_test_listings_2

[<Job Data Scientist / Senior Data Scientist Basware>,
 <Job Data Engineer - Tietoevry Care Data and Analytics Tietoevry>,
 <Job Data Scientist MedEngine>,
 <Job Data Engineer - Tietoevry Care Data and Analytics Tietoevry>,
 <Job Data Platform Engineer Scandit>,
 <Job Data Engineer (Level Up) Loihde Advance>,
 <Job ETL Specialist Gazelle Global>,
 <Job Data Engineer - Tietoevry Care Data and Analytics Tietoevry>,
 <Job Data Engineer - Tietoevry Care Data and Analytics Tietoevry>,
 <Job IT-asiantuntija Anfra Oy>,
 <Job (Senior) Data Engineer - Tietoevry Tech Services Tietoevry>,
 <Job Machine Learning Engineer - MLOps Wolt>,
 <Job Data Engineer Suomen Palloliitto - Football Association of Finland>,
 <Job Quantitative Risk Analyst Nordea>,
 <Job Cloud & Enterprise Architect Samlink – A Kyndryl Company>,
 <Job Data Engineer Epical>]